# Clasificación multiclase softmax

La clasificación softmax es una generalización de la regresión logística binaria, donde en vez de dividir o separar mediante un valor delimitante se asignan valores de probabilidad mediante la 'normalización' de valores de salida o las clases mediante la funcion *softmax* o también conocido como *función exponencial normalizada*

La función se define de l

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt

## Desarrollo de teoria

## Funciones de preparacion de datos y entrenamiento

In [2]:
def oneHotEncoder(target):
    n_classes = np.unique(target).shape[0]
    y_encode = np.zeros((target.shape[0], n_classes))
    for idx, val in enumerate(y):
        y_encode[idx, val] = 1.0
    return y_encode

In [3]:
def sigmoid(z):
    return np.exp(z) / np.sum(np.exp(z), axis = 1, keepdims = True)

In [4]:
def split_train_test(data, target, train_q = 35):
    if (train_q < 0) or (train_q > 49): return 0, 0, 0, 0
    
    data_train = np.concatenate((data[0:train_q], data[50:(train_q + 50)], data[100:(train_q + 100)]))
    data_test = np.concatenate ((data[train_q:50], data[(train_q + 50):100], data[(train_q + 100):150]))
    target_train = np.concatenate((target[0:train_q], target[50:(train_q + 50)], target[100:(train_q + 100)]))
    target_test = np.concatenate ((target[train_q:50], target[(train_q + 50):100], target[(train_q + 100):150]))
    
    return data_train, data_test, target_train, target_test

In [5]:
def model_fit(data, target, eta=0.55, iterations=100000):
    m = len(target)
    print(f'target: ', m)
    
    theta = np.random.randn(data.shape[1], target.shape[1])
    print(f'theta: \n', theta)
    
    for i in range(iterations):
        gradients = (1/m) * (data.T @ (sigmoid(data @ theta) - target))
        theta = theta - eta * gradients
    print(f'\n',theta)
    return theta

In [6]:
def model_test(sepal_length, sepal_width, petal_length, petal_width, weights):
    list1 = [0, 0, 0]
    
    for i in range(len(list1)):
        a0 = weights.T[i][0]
        a1 = weights.T[i][1]
        a2 = weights.T[i][2]
        a3 = weights.T[i][3]
        a4 = weights.T[i][4]
        list1[i] = np.exp(a0 + a1 * sepal_length + a2 * sepal_width + a3 * petal_length + a4 * petal_width)
    maxP = np.argmax([z / sum(list1) for z in list1])
    pred = [0, 0, 0]
    pred[maxP] = 1
    return pred
    #return [z / sum(list1) for z in list1]

In [7]:
def model_predict(data, target, weights):
    predict_list = []
    test_list = []
    for i in data:
        predict_list.append(np.argmax(model_test(i[0], i[1], i[2], i[3], weights)))
    for j in target:
        test_list.append(np.argmax(j))
    num = 0
    for k in range(len(predict_list)):
        if predict_list[k] == test_list[k]: num = num +1
        
    final_list = np.array([predict_list, test_list], ndmin=2)
    effi = num/len(predict_list)
    return final_list, effi

## Base de datos: Iris

In [8]:
iris = datasets.load_iris()

X = iris['data']
y = iris['target']

X_c = np.c_[np.ones((len(X), 1)), X]
y_c = oneHotEncoder(y)

## Preparación de datos, división en split

In [9]:
X_train, X_test, y_train, y_test = split_train_test(X_c, y_c)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(105, 5) (45, 5) (105, 3) (45, 3)


## Entrenamiento

In [10]:
a = model_fit(X_train, y_train)

target:  105
theta: 
 [[ 0.89524332  0.12089245  2.46052881]
 [-0.24469202 -0.1778048   1.42586629]
 [-0.78005276  1.63099264 -0.71565783]
 [-1.27720575 -1.15385804 -0.17571277]
 [ 0.88244162  0.89607265 -0.74929259]]

 [[  2.59196655  19.01912074 -18.13442271]
 [  3.55425772  -0.09660858  -2.45427968]
 [  7.08163481  -0.22230291  -6.72404985]
 [-10.25620885  -0.61067148   8.26010376]
 [ -3.73665257  -5.52664846  10.29252271]]


## Prediccón

In [11]:
predictions, efficiency = model_predict(X_test, y_test, a)

print(predictions, efficiency)

[[1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 2 2 0 2 0 2 2 0 0 2 2 2 2 0 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2]] 0.35555555555555557
